In [1]:
# Test to see why imports are not working as expected
import sys 
print(sys.version)

3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:41:52) [Clang 15.0.7 ]


In [2]:
!pip list

Package                       Version
----------------------------- -----------
absl-py                       1.2.0
anyio                         3.6.1
appnope                       0.1.3
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.0.8
astunparse                    1.6.3
attrs                         22.1.0
Babel                         2.10.3
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.12.2
bleach                        5.0.1
brotlipy                      0.7.0
cached-property               1.5.2
cachetools                    5.2.0
certifi                       2022.9.24
cffi                          1.15.1
charset-normalizer            2.1.1
colorama                      0.4.5
conda                         22.9.0
conda-package-handling        1.8.1
contourpy                     1.0.7
cryptography                  37.0.4
cycler                        0.11.0
debugpy

Power of Semantics Search combined with Elastic Search | ML on ELK

In [5]:
try:
    import json
    import os
    import uuid

    import pandas as pd
    import numpy as np

    import elasticsearch
    
    import ast
    
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers
    from sentence_transformers import SentenceTransformer, util
    from tqdm import tqdm
    from dotenv import load_dotenv
    load_dotenv("secret.env")
except Exception as e:
    print("Some Modules are Missing :{}".format(e))

In [25]:
df = pd.read_csv('../course_vector_generator/job_vector.csv')

df["status"] = "published"
    
#df['vectors'] = ast.literal_eval(df['vectors'])   
df['vectors'] = df.vectors.apply(lambda x: ast.literal_eval(str(x)))

df['vectors'][0]
#df['vectors_n'][0]
#vector_arrays = df['vectors'].tolist()

#vector_arrays

[-0.032982103526592255,
 0.03393172472715378,
 -0.08094385266304016,
 0.038092732429504395,
 -0.09114564955234528,
 -0.023579811677336693,
 0.046846650540828705,
 0.05419328063726425,
 0.05766279250383377,
 0.036884941160678864,
 -0.020067142322659492,
 -0.05997448414564133,
 -0.0735679566860199,
 0.047451939433813095,
 -0.0874614343047142,
 -0.026564206928014755,
 0.02579645998775959,
 0.0023600549902766943,
 -0.03271115943789482,
 -0.011985016986727715,
 0.058256443589925766,
 -0.052371080964803696,
 0.06312091648578644,
 -0.0503850020468235,
 -0.04738481715321541,
 -0.005381381139159203,
 -0.014134641736745834,
 0.041706766933202744,
 0.017755696550011635,
 0.00943952240049839,
 -0.01637670025229454,
 0.012050294317305088,
 0.09865079820156097,
 0.006726446095854044,
 0.03071448765695095,
 0.14239321649074554,
 0.008073274046182632,
 0.007256313227117062,
 0.03586052358150482,
 0.014134548604488373,
 0.009634995833039284,
 0.023912085220217705,
 -0.01924419216811657,
 -0.05382756143

## Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "900x3MM9iJ6aLSxkpeqfRpLe"

# Found in the 'Manage Deployment' page
CLOUD_ID = "My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRiMmYwN2IyNmUyYTE0MzlkYThhNjQ0ZmVhNDZmYzAzNSQ5ZTAxZDAyMzY0Mjk0MGU1YWQ1MzJlMThhNDAyMzljMA=="
'''
# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Successful response!
client.info()
'''

In [6]:
class Reader(object):

    def __init__(self, file_name):
        self.file_name = file_name

    def run(self):

        df = pd.read_csv(self.file_name, chunksize=3000)
        df = next(df)
        df = df.fillna("")
        return df


In [7]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences, convert_to_tensor=True)
        #encod_list = sentence_embeddings.numpy()
        _ = list(sentence_embeddings.flatten())
        encod_np_array = np.array(_)
        encod_list = encod_np_array.tolist()
        return encod_list


In [8]:
class ElasticSearchImports(object):
    def __init__(self, df, index_name='posting'):
        self.df = df
        self.index_name = index_name
        self.es = Elasticsearch("https://localhost:9200", basic_auth=('elastic', 'OKzimtr0od5trZ0j=DUm'))
        
        '''
        self.es = client = Elasticsearch(
            cloud_id=CLOUD_ID,
            basic_auth=("elastic", ELASTIC_PASSWORD)
        )
        '''

        #self.es = Elasticsearch(timeout=600,hosts=os.getenv("ENDPOINT"), basic_auth=('username', 'password'))
        #self.es = Elasticsearch(timeout=600,hosts=os.getenv("ENDPOINT"))
        #es = Elasticsearch( "https://localhost:9200",ca_certs="/path/to/http_ca.crt", basic_auth=("elastic", ELASTIC_PASSWORD))

    def run(self):

        elk_data = self.df.to_dict("records")

        for job in elk_data:
            try:self.es.index(index=self.index_name,body=job)
            except Exception as e:pass

        return True

# Converting column to Vector Enbeddings

In [9]:
helper = Reader(file_name="data job posts.csv")
df = helper.run()

In [10]:
df["jobpost"]

0       AMERIA Investment Consulting Company\r\nJOB TI...
1       International Research & Exchanges Board (IREX...
2       Caucasus Environmental NGO Network (CENN)\r\nJ...
3       Manoff Group\r\nJOB TITLE:  BCC Specialist\r\n...
4       Yerevan Brandy Company\r\nJOB TITLE:  Software...
                              ...                        
2995    British American Tobacco\r\nTITLE:  Merchandis...
2996    ArmenTel CJSC\r\nTITLE:  Rating and Billing Co...
2997    British American Tobacco (BAT)\r\nTITLE:  HoRe...
2998    Partner Organization of Career Center\r\nTITLE...
2999    Boomerang Software LLC\r\nTITLE:  Lawyer\r\nST...
Name: jobpost, Length: 3000, dtype: object

In [ ]:
tqdm.pandas()
helper_token = Tokenizer()
df["vectors"] = df["jobpost"].progress_apply(helper_token.get_token)

 53%|████████████████████▊                  | 1597/3000 [03:28<02:38,  8.83it/s]

In [ ]:
df

In [ ]:
df["vectors"]

In [ ]:
df.vectors[0]

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, algorithm='auto')
knn.fit(df['vectors'].to_numpy().tolist())

def get_neighbors(vector):
    return knn.kneighbors([vector], 10, return_distance=False)



In [ ]:
def get_k_neighbours(df,number_of_neighbours):

    def get_knn(df):
        vector_arrays = df['vectors'].to_numpy().tolist()
        return NearestNeighbors().fit(vector_arrays)        

    def get_token():
        helper_token = Tokenizer()
        INPUT = input("Enter the Input Query ")
        token_vector = helper_token.get_token(INPUT)
        return token_vector

    def flatten_neighbour_list(nb_indexes):
        nb_list = nb_indexes.tolist()
        return [item for sublist in nb_list for item in sublist]        

    knn = get_knn(df)
    vector = get_token()
    nb_indexes = knn.kneighbors([vector],number_of_neighbours,return_distance=False)
    nb_indexes = flatten_neighbour_list(nb_indexes)
    
    title_arrays = df['Title'].to_numpy().tolist()
    post_arrays = df['jobpost'].to_numpy().tolist()
    return_array = []
    for num in nb_indexes:
        return_array.append([title_arrays[num], post_arrays[num].replace('\r', '').replace('\n', '')])
    
    
    return return_array

In [ ]:
get_k_neighbours(df , 10)